# TICA + clusterization of mixed solvent simulations

In this document you can find the process explained in *"NAME OF ARTICLE"*
This document is an adaptation of the PyEMMA tutorials to this framework. If you want to check the original source please refer to: http://www.emma-project.org/latest/tutorial.html

To perform these calculations you will need to have installed the following software/libraries:

**Part 1 (Python)**
* pyemma http://www.emma-project.org/latest/INSTALL.html
* numpy
* matplotlib
* mdtraj https://www.mdtraj.org/1.9.5/installation.html

**Part 2 (R)**
* ggplot2
* gridextra
* grid
* dplyr
* ggpointdensity
* ggsci
* cowplot

In this document you will find the first part of the process, which in consists in dimensionality reduction based on TICA

## 1. Load python libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pyemma

# Define scoring function for our features
def score_cv(data, dim, lag, number_of_splits=10, validation_fraction=0.5):
    """Compute a cross-validated VAMP2 score.
    
    We randomly split the list of independent trajectories into
    a training and a validation set, compute the VAMP2 score,
    and repeat this process several times.
    
    Parameters
    ----------
    data : list of numpy.ndarrays
        The input data.
    dim : int
        Number of processes to score; equivalent to the dimension
        after projecting the data with VAMP2.
    lag : int
        Lag time for the VAMP2 scoring.
    number_of_splits : int, optional, default=10
        How often do we repeat the splitting and score calculation.
    validation_fraction : int, optional, default=0.5
        Fraction of trajectories which should go into the validation
        set during a split.
    """
    # we temporarily suppress very short-lived progress bars
    from pyemma.util.contexts import settings
    with settings(show_progress_bars=False):
        nval = int(len(data) * validation_fraction)
        scores = np.zeros(number_of_splits)
        for n in range(number_of_splits):
            ival = np.random.choice(len(data), size=nval, replace=False)
            vamp = pyemma.coordinates.vamp(
                [d for i, d in enumerate(data) if i not in ival], lag=lag, dim=dim)
            scores[n] = vamp.score([d for i, d in enumerate(data) if i in ival])
    return scores

## 2. Load data: PDB and MD trajectories

Please remember to align your trajectories before performing any kind of analysis.
You can use *TICA_Mixed_Solvent_fitting.py*

In [ ]:
pdb = 'XYZ.pdb' #Reference PDB
import glob
files = []
for file in glob.glob('XYZ*.xtc'): #Load trajectory or a series of trajectories. In this example is done as the latter
    files.append(file)
files

In [ ]:
files = sorted(files)
files

## 3. Load features

In [ ]:
#Torsions
torsions_feat = pyemma.coordinates.featurizer(pdb)
torsions_feat.add_backbone_torsions(cossin=True, periodic=False)
torsions_data = pyemma.coordinates.load(files, features=torsions_feat)

#Positions
positions_feat = pyemma.coordinates.featurizer(pdb)
positions_feat.add_selection(positions_feat.select_Backbone())
positions_data = pyemma.coordinates.load(files, features=positions_feat)

#Distances between Key residues/ other?
distances_feat = pyemma.coordinates.featurizer(pdb)
distance_indices = distances_feat.select('index XXX YYY ZZZ') #Identify which atom indices correspond to the ones of interest
distances_feat.add_distances(distance_indices, periodic=False)
distances_data = pyemma.coordinates.load(files, features=distances_feat)



In [ ]:
distances_feat.describe() #We check that we have selected to correct atoms

## 4.1 TICA for backbone torsions as a feature

In [ ]:
tica = pyemma.coordinates.tica(torsions_data, lag=XY, dim=4) #choose a lag time
tica_output = tica.get_output()
tica_concatenated = np.concatenate(tica_output)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(
    tica_concatenated,
    ax=axes[0],
    feature_labels=['IC1', 'IC2', 'IC3', 'IC4'],
    ylog=True)
pyemma.plots.plot_density(*tica_concatenated[:, :2].T, ax=axes[1], logscale=True)
axes[1].set_xlabel('IC 1')
axes[1].set_ylabel('IC 2')
fig.tight_layout()

### Save TICA data for later use

In [ ]:
np.savetxt("tica_torsions.csv", tica_concatenated, delimiter=",", header='TIC1,TIC2,TIC4,TIC4')

### Get extremes for easy visualization on the TICA motions

In [ ]:
tica.get_output(dimensions=[0])
order = np.argsort(np.concatenate(tics)[:,0])[0:50]
per_traj_indices = np.concatenate([np.array((np.ones(l, dtype=int)*i, np.arange(l))).T for i,l in enumerate(tica.trajectory_lengths())])
pyemma.coordinates.save_traj(files, per_traj_indices[order], 'TIC1_extreme_negative_torsions.xtc', top=pdb)

In [ ]:
tics = tica.get_output(dimensions=[1])
order = np.argsort(np.concatenate(tics)[:,0])[0:50]
per_traj_indices = np.concatenate([np.array((np.ones(l, dtype=int)*i, np.arange(l))).T for i,l in enumerate(tica.trajectory_lengths())])
pyemma.coordinates.save_traj(files, per_traj_indices[order], 'TIC2_extreme_negative_torsions.xtc', top=pdb)

## 4.2 TICA for backbone coordinates as a feature

In [ ]:
tica = pyemma.coordinates.tica(positions_data, lag=XY, dim=4) #choose a lag time
tica_output = tica.get_output()
tica_concatenated = np.concatenate(tica_output)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(
    tica_concatenated,
    ax=axes[0],
    feature_labels=['IC1', 'IC2', 'IC3', 'IC4'],
    ylog=True)
pyemma.plots.plot_density(*tica_concatenated[:, :2].T, ax=axes[1], logscale=True)
axes[1].set_xlabel('IC 1')
axes[1].set_ylabel('IC 2')
fig.tight_layout()

### Save TICA data for later use

In [ ]:
np.savetxt("tica_coord.csv", tica_concatenated, delimiter=",", header='TIC1,TIC2,TIC4,TIC4')

### Get extremes for easy visualization on the TICA motions

In [ ]:
tics = tica.get_output(dimensions=[0])
order = np.argsort(np.concatenate(tics)[:,0])[0:50]
per_traj_indices = np.concatenate([np.array((np.ones(l, dtype=int)*i, np.arange(l))).T for i,l in enumerate(tica.trajectory_lengths())])
pyemma.coordinates.save_traj(files, per_traj_indices[order], 'TIC1_extreme_negative_coord.xtc', top=pdb)

In [ ]:
tics = tica.get_output(dimensions=[1])
order = np.argsort(np.concatenate(tics)[:,0])[0:50]
per_traj_indices = np.concatenate([np.array((np.ones(l, dtype=int)*i, np.arange(l))).T for i,l in enumerate(tica.trajectory_lengths())])
pyemma.coordinates.save_traj(files, per_traj_indices[order], 'TIC2_extreme_negative_coord.xtc', top=pdb)

## 4.3 TICA for distances between atoms

In [ ]:
tica = pyemma.coordinates.tica(distances_data, lag=XY, dim=4) #choose a lag time
tica_output = tica.get_output()
tica_concatenated = np.concatenate(tica_output)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(
    tica_concatenated,
    ax=axes[0],
    feature_labels=['IC1', 'IC2', 'IC3','IC4'],
    ylog=True)
pyemma.plots.plot_density(*tica_concatenated[:, :2].T, ax=axes[1], logscale=True)
axes[1].set_xlabel('IC 1')
axes[1].set_ylabel('IC 2')
fig.tight_layout()

### Save TICA for later use

In [ ]:
np.savetxt("tica_distances.csv", tica_concatenated, delimiter=",", header='TIC1,TIC2,TIC4,TIC4')

### Get extremes for easy visualization on the TICA motions

In [ ]:
tics = tica.get_output(dimensions=[0])
order = np.argsort(np.concatenate(tics)[:,0])[0:50]
per_traj_indices = np.concatenate([np.array((np.ones(l, dtype=int)*i, np.arange(l))).T for i,l in enumerate(tica.trajectory_lengths())])
pyemma.coordinates.save_traj(files, per_traj_indices[order], 'TIC1_extreme_negative_distances.xtc', top=pdb)

In [ ]:
tics = tica.get_output(dimensions=[1])
order = np.argsort(np.concatenate(tics)[:,0])[0:50]
per_traj_indices = np.concatenate([np.array((np.ones(l, dtype=int)*i, np.arange(l))).T for i,l in enumerate(tica.trajectory_lengths())])
pyemma.coordinates.save_traj(files, per_traj_indices[order], 'TIC2_extreme_negative_distances.xtc', top=pdb)

## 5. Clusterize to calculate Implied Timescales

In [ ]:
cluster = pyemma.coordinates.cluster_kmeans(
    tica_output, k=100, max_iter=50, stride=10, fixed_seed=1)
dtrajs_concatenated = np.concatenate(cluster.dtrajs)

fig, ax = plt.subplots(figsize=(4, 4))
pyemma.plots.plot_free_energy(
    *tica_concatenated[:, :2].T, ax=ax, cbar=False, legacy=False)
ax.scatter(*cluster.clustercenters[:, :2].T, s=15, c='k')
ax.set_xlabel('IC 1')
ax.set_ylabel('IC 2')
fig.tight_layout()

## 6. Calculate Implied Timescales

In [ ]:
its = pyemma.msm.its(cluster.dtrajs, lags=XY, nits=10, errors='bayes')
pyemma.plots.plot_implied_timescales(its, units='ns', dt=0.2);